### Project - Historical Product Demand

#### Problem definition : 

- As stated in the notes for this dataset - the problem is that this customer has many warehouses across the globe across many product IDs. 
- It takes one month to ship product to a warehouse location.

#### Goal : 
- To create accurate forecasting across warehouses and product ID so inventory can be optimized - resulting in no loss in sales and/or no excess production. 

#### How is it being solved today : 
- Since we dont have access to the original stakeholders, the assumption is a manual/instinctive forecasting being applied which may be resulting in inefficiencies(Gaps). 

#### Metrics for success : 
- By hypothesis testing we will confirm a 95% confidence interval between the distributions of the actual test values (lets say 10% of the tail end of the data) and our predicted results. 
- We will also compare the results to a Simple Moving Average and/or Exponential Weighted Moving Average
- We will also report the Mean Absolute Error, Mean Squared Error and Root Mean Squared Error in context of the mean of the distribution.

#### Actions to be taken based on this work : 
- Optimize inventory at each location using forecasts.
- Any further actions that become evident during Exploratory Data Analysis or other steps. 

#### Scope of the project :
- Data Collection - Not Required
- Analysis - Required
- Observation and Reporting - Required
- Prediction - Required
- Actionalable insights - Required. Aim to identify seasonal trends, demand spikes and underperforming products.
- Deployment - Not Required
- Retraining Pipelines - Required
- Visualisations - Required

#### Timelines : 
- High Level Timeline estimate - Not Required for Sample project
- Granular Task and Timelines breakdown - Not Required for Sample project

#### Ethical & Regulatory considerations for this project : None

#### Data Collection  : Not required for Sample project
- Integration of different data sources
- Data Privacy and Compliance
- Data Storage and Security
- Data Accessibility

#### Project Steps	
- Data Exploration, Cleaning and Preparation
- Feature Engineering
- Feature Selection
- Model Building
- Model Evaluation

#### Deployment	: Not Required for Sample project
- Model Deployment
- Version Control
- Model Monitoring and Maintenance Plan
- Scalability Considerations
- Automated Testing

#### Communicate and Reporting
- Report Findings - Required
- Stakeholder Presentations - Not required for Sample project
- Create Dashboards for interactive reporting - Not required for Sample project
- User Training - Not required for Sample project
	
#### Documentation	
- Project steps and methodologies, parameters, deployment details - In Notebook, during project development
- Code Documentation during development - In Notebook, during project development
- Data Lineage Documentation - Not Required
- Model Explanation and Interpretability - In Notebook, during project development
	
#### Review	
- Review Metrics of project - Via confirmation of Hypothesis testing, vs baseline SMA and EWMA and common error metrics MAE, MSE and RMSE
- Lessons learned - In Notebook, after project development
- Feedback Collection - Currently not required. 
- Follow up actions - Currently not required
- Model Decommissioning Plan - Currently not required


### Project Approach: 

- Since there are many warehouses and projects, we need to forecast orders demand for each product at each location. 
- We will break down the dataset into separate dataframes for each warehouse and each product and perform predictions on each. 
- We need to ensure the code is modular so that in future forecasting can be applied to the entire data again or to an individual warehouse or product. 
- We will be attempting ARMA, ARIMA and SARIMA models to make our predictions. Failing good results we may try LSTM models but that option is not preferable due to explainability and interpretability. 
###### Note that at some later point in the business, depending on the success of this project, it may be a good idea to try VAR and VARMA models between different locations of warehouses. The assumption being that proximity between warehouses may affect each other. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import ks_2samp
import joblib
from fpdf import FPDF
import os
import warnings

warnings.filterwarnings('ignore')

# Load the master DataFrame from a CSV file
file_name = 'Historical Product Demand.csv'
master = pd.read_csv(file_name)

# Function to preprocess and aggregate data
def preprocess_and_aggregate(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Order_Demand'] = df['Order_Demand'].str.strip('()').astype(float)
    df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()
    aggregated_df = df.groupby(['Month', 'Product_Code', 'Warehouse', 'Product_Category'])['Order_Demand'].sum().reset_index()
    return aggregated_df

# Function to create a time series with all months filled
def create_filled_time_series(df):
    min_date = df['Month'].min()
    max_date = df['Month'].max()
    all_months = pd.date_range(start=min_date, end=max_date, freq='MS')
    df.set_index('Month', inplace=True)
    df = df.reindex(all_months, fill_value=0).rename_axis('Month').reset_index()
    return df

# Preprocess and aggregate the data
preprocessed_master = preprocess_and_aggregate(master)

# Function to run full analysis on a given dataframe
def run_full_analysis(df, title):
    # Check if DataFrame has sufficient length
    if len(df) < 24:
        print(f"Skipping {title}: insufficient data length ({len(df)} observations).")
        return None

    product_code = title.split(' ')[0]
    warehouse = title.split(' ')[2]

    # Plot Order Demand
    plt.figure(figsize=(10, 6))
    plt.plot(df['Month'], df['Order_Demand'])
    plt.title(f"Order Demand for {title}")
    plt.xlabel('Date')
    plt.ylabel('Order_Demand')
    plt.savefig(f'Order_Demand_{product_code}_{warehouse}.png')
    plt.show()

    # Seasonal Decomposition
    try:
        decomposition = seasonal_decompose(df['Order_Demand'], model='additive', period=12)
        decomposition.plot()
        plt.savefig(f'Seasonal_Decomposition_{product_code}_{warehouse}.png')
        plt.show()
    except ValueError as e:
        print(f"Error in seasonal decomposition for {title}: {e}")
        return None

    # Determine appropriate lags for ACF and PACF plots
    nlags = min(40, max(1, len(df) // 2 - 1))

    # ACF and PACF plots
    try:
        plt.figure(figsize=(10, 6))
        plot_acf(df['Order_Demand'], lags=nlags)
        plt.savefig(f'ACF_{product_code}_{warehouse}.png')
        plt.show()

        plt.figure(figsize=(10, 6))
        plot_pacf(df['Order_Demand'], lags=nlags)
        plt.savefig(f'PACF_{product_code}_{warehouse}.png')
        plt.show()
    except ValueError as e:
        print(f"Error in ACF/PACF plots for {title}: {e}")
        return None

    # ADF test report
    def adf_test(series, title=''):
        result = adfuller(series.dropna(), autolag='AIC')
        labels = ['ADF test statistic', 'p-value', '# lags used', '# observations']
        out = pd.Series(result[0:4], index=labels)
        for key, val in result[4].items():
            out[f'critical value ({key})'] = val

        print(f'Augmented Dickey-Fuller Test: {title}')
        print(out.to_string())

        if result[1] <= 0.05:
            conclusion = "Strong evidence against the null hypothesis. Reject the null hypothesis. Data has no unit root and is stationary."
        else:
            conclusion = "Weak evidence against the null hypothesis. Fail to reject the null hypothesis. Data has a unit root and is non-stationary."
        
        print(conclusion)
        return out, conclusion

    adf_results, adf_conclusion = adf_test(df['Order_Demand'], title=f'{title}')

    # Split train and test
    train_size = int(len(df) * 0.8)
    train, test = df[:train_size], df[train_size:]

    if train['Order_Demand'].isnull().all() or test['Order_Demand'].isnull().all():
        print(f"Insufficient data for {title} after splitting. Skipping this combination.")
        return None

    # Fit SARIMA model with specified parameters
    order = (0, 0, 0)
    seasonal_order = (0, 2, 3, 12)
    try:
        model = SARIMAX(train['Order_Demand'], order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
    except Exception as e:
        print(f"Error fitting SARIMAX model for {title}: {e}")
        return None

    # Forecast for the test period
    forecast = model_fit.get_forecast(steps=len(test))
    forecast_mean = forecast.predicted_mean  # These are the predicted values for the test period
    forecast_ci = forecast.conf_int()  # Confidence intervals for the predicted values

    # Forecast for the next 12 months
    forecast_12_months = model_fit.get_forecast(steps=12)
    forecast_12_mean = forecast_12_months.predicted_mean  # Predicted values for the next 12 months
    forecast_12_ci = forecast_12_months.conf_int()  # Confidence intervals for the next 12 months

    # Forecast for the next 1 month
    forecast_1_month = model_fit.get_forecast(steps=1)
    forecast_1_mean = forecast_1_month.predicted_mean.iloc[0]  # Predicted value for the next 1 month

    # Metrics
    mae = mean_absolute_error(test['Order_Demand'], forecast_mean)
    mse = mean_squared_error(test['Order_Demand'], forecast_mean)
    rmse = np.sqrt(mse)
    mean_val = np.mean(test['Order_Demand'])
    std_val = np.std(test['Order_Demand'])
    mse_mean_ratio = mse / mean_val
    rmse_std_ratio = rmse / std_val

    # Kolmogorov-Smirnov test
    ks_stat, ks_p_value = ks_2samp(test['Order_Demand'], forecast_mean)
    print(f"KS Test Value: {ks_stat}, KS p-Value: {ks_p_value}")

    # Determine status based on KS test p-value
    if ks_p_value >= 0.7:
        status = 'deploy'
    elif 0.3 <= ks_p_value < 0.7:
        status = 'deploy and monitor'
    else:
        status = 'retrain'

    # Save model if status is deploy or deploy and monitor
    if status in ['deploy', 'deploy and monitor']:
        model_name = f'{warehouse}{product_code[8:]}261222.pkl'
        joblib.dump(model_fit, os.path.join('HPD_models', model_name))

    # Metrics DataFrame
    metrics = {
        'Product_code': product_code,
        'Warehouse': warehouse,
        '1 Month Forecast': forecast_1_mean,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'Mean': mean_val,
        'STD': std_val,
        'MSE/Mean': mse_mean_ratio,
        'RMSE/STD': rmse_std_ratio,
        'KS Test Value': ks_stat,
        'KS p-Value': ks_p_value,
        'Status': status
    }
    metrics_df = pd.DataFrame(metrics, index=[0])

    # Plot Actual vs Predicted
    plt.figure(figsize=(10, 6))
    plt.plot(train['Month'], train['Order_Demand'], label='Train')
    plt.plot(test['Month'], test['Order_Demand'], label='Test')
    plt.plot(test['Month'], forecast_mean, label='Predicted')
    plt.fill_between(test['Month'], forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='k', alpha=0.1)
    plt.legend()
    plt.title('Actual vs Predicted')
    plt.savefig(f'Actual_vs_Predicted_{product_code}_{warehouse}.png')
    plt.show()

    # Calculate and Plot Simple Moving Average and Exponential Weighted Moving Average
    sma = df['Order_Demand'].rolling(window=12).mean()
    ewma = df['Order_Demand'].ewm(span=12, adjust=False).mean()

    plt.figure(figsize=(10, 6))
    plt.plot(test['Month'], test['Order_Demand'], label='Actual Test')
    plt.plot(test['Month'], forecast_mean, label='Predicted')
    plt.plot(df['Month'], sma, label='Simple Moving Average')
    plt.plot(df['Month'], ewma, label='Exponential Weighted Moving Average')
    plt.legend()
    plt.title('Test, Predicted with SMA and EWMA')
    plt.savefig(f'Test_vs_SMA_EWMA_{product_code}_{warehouse}.png')
    plt.show()

    # Plot Forecast
    plt.figure(figsize=(10, 6))
    plt.plot(df['Month'], df['Order_Demand'], label='History')
    plt.plot(pd.date_range(start=test['Month'].iloc[-1] + pd.DateOffset(months=1), periods=12, freq='MS'), forecast_12_mean, label='Forecast')
    plt.fill_between(pd.date_range(start=test['Month'].iloc[-1] + pd.DateOffset(months=1), periods=12, freq='MS'), forecast_12_ci.iloc[:, 0], forecast_12_ci.iloc[:, 1], color='k', alpha=0.1)
    plt.legend()
    plt.title('12-Month Forecast')
    plt.savefig(f'12_Month_Forecast_{product_code}_{warehouse}.png')
    plt.show()

    return metrics_df, adf_results, adf_conclusion

# Initialize a list to collect metrics dataframes
all_metrics = []
monthly_forecast = {}

# Initialize counter
counter = 1

# Process and store each dataframe
for (product_code, warehouse), group in preprocessed_master.groupby(['Product_Code', 'Warehouse']):
    filled_df = create_filled_time_series(group)
    result = run_full_analysis(filled_df, f'{product_code} in {warehouse}...Sequential Number {counter}')
    if result is not None:
        metrics_df, adf_results, adf_conclusion = result
        all_metrics.append(metrics_df)
        if product_code not in monthly_forecast:
            monthly_forecast[product_code] = {}
        monthly_forecast[product_code][warehouse] = metrics_df['1 Month Forecast'].values[0]
    counter += 1

# Concatenate all metrics dataframes
final_metrics_df = pd.concat(all_metrics, ignore_index=True)

# Calculate summary statistics
total_values = len(final_metrics_df)
deploy_values = len(final_metrics_df[final_metrics_df['Status'] == 'deploy'])
deploy_and_monitor_values = len(final_metrics_df[final_metrics_df['Status'] == 'deploy and monitor'])
retrain_values = len(final_metrics_df[final_metrics_df['Status'] == 'retrain'])

summary_statistics = {
    'Total Values': total_values,
    'Deploy Values': deploy_values,
    'Deploy Values %': deploy_values / total_values,
    'Deploy and Monitor Values': deploy_and_monitor_values,
    'Deploy and Monitor Values %': deploy_and_monitor_values / total_values,
    'Retrain Values': retrain_values,
    'Retrain Values %': retrain_values / total_values
}

summary_statistics_df = pd.DataFrame(summary_statistics, index=[0])

# Create the monthly forecast table
monthly_forecast_df = pd.DataFrame(monthly_forecast).T
monthly_forecast_df['Total'] = monthly_forecast_df.sum(axis=1)

# Save the reports to PDF
reports_dir = 'Reports'
if not os.path.exists(reports_dir):
    os.makedirs(reports_dir)

# Function to generate a PDF report
def generate_pdf_report(product_code, warehouse, adf_results, adf_conclusion):
    pdf = FPDF()
    pdf.add_page()

    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=f"Sample Report for Product Code: {product_code} Warehouse: {warehouse}", ln=True, align='C')

    # Add plots
    for plot_name in ['Order_Demand', 'Seasonal_Decomposition', 'ACF', 'PACF', 'Actual_vs_Predicted', 'Test_vs_SMA_EWMA', '12_Month_Forecast']:
        pdf.image(f'{plot_name}_{product_code}_{warehouse}.png', x=10, w=180)
        pdf.ln(85)

    # Add ADF test results
    pdf.set_font("Arial", size=10)
    pdf.cell(200, 10, txt="ADF Test Results", ln=True, align='L')
    for label, value in adf_results.items():
        pdf.cell(200, 10, txt=f"{label}: {value}", ln=True, align='L')
    pdf.cell(200, 10, txt=adf_conclusion, ln=True, align='L')

    # Save the PDF
    pdf.output(os.path.join(reports_dir, f'Sample Report for Product Code {product_code} Warehouse {warehouse}.pdf'))

# Generate the sample report for the first product_code and warehouse
if all_metrics:
    first_metrics_df = all_metrics[0]
    generate_pdf_report(first_metrics_df['Product_code'].iloc[0], first_metrics_df['Warehouse'].iloc[0], adf_results, adf_conclusion)

# Save final metrics and summary statistics to PDF
final_metrics_df.to_csv(os.path.join(reports_dir, 'Final Metrics.csv'))
summary_statistics_df.to_csv(os.path.join(reports_dir, 'Model Performance Summaries.csv'))
monthly_forecast_df.to_csv(os.path.join(reports_dir, 'Monthly Forecast.csv'))

# Output the final metrics dataframe and summary statistics
print(final_metrics_df)
print(summary_statistics_df)
print(monthly_forecast_df)


### Review: 






##### Notes and observations: 
- It may be a good idea to run VAR or VARMA models on the data with multiple warehouse locations orders being the multivariate feature to be forecast. The assumption being that proximity between warehouses may affect each other. 